In [1]:
import gzip
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import netCDF4 as nc
import match
from netCDF4 import Dataset
import xarray as xr
import os 
import pandas as pd  
from datetime import datetime

# 生成文件目录

In [3]:
start_year = 2021
end_year = 2021

folder_path = 'D:/data/SST/2021_/'  # 文件夹路径
file_names = os.listdir(folder_path)  
len(file_names)  

365

In [4]:
file_names[0],file_names[-1]

('20210101120000-UKMO-L4_GHRSST-SSTfnd-OSTIA-GLOB_REP-v02.0-fv02.0.nc',
 '20211231120000-UKMO-L4_GHRSST-SSTfnd-OSTIA-GLOB_REP-v02.0-fv02.0.nc')

In [5]:
# 打开所有nc文件
datasets = [xr.open_dataset(folder_path + file_name) for file_name in file_names]
# 合并多个数据集
merged_dataset = xr.concat(datasets, dim='time')  # 'time'是合并的维度

In [5]:
#merged_dataset.to_netcdf('D:/data/SST/SST_2013-2014_concat.nc')

# 读取合并数据

In [6]:
for var in merged_dataset.variables.keys():
    data = merged_dataset.variables[var][:].data
    print(var,data.shape)

time (365,)
lat (3600,)
lon (7200,)
analysed_sst (365, 3600, 7200)
analysis_error (365, 3600, 7200)
sea_ice_fraction (365, 3600, 7200)
mask (365, 3600, 7200)


In [7]:
# nc = Dataset('D:/data/SST/SST_2011-2012_concat.nc')
# print(nc.variables.keys())

# for var in nc.variables.keys():
#     data = nc.variables[var][:].data
#     print(var,data.shape)

In [8]:
sst = merged_dataset.variables['analysed_sst'][:].data
#mask = merged_dataset.variables['mask'][0].data

lat   =  merged_dataset.variables['lat'][:].data
lon   =  merged_dataset.variables['lon'][:].data

In [9]:
sst[sst == -32768] = np.nan

In [10]:
start_year = 2019
end_year = 2019
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year,12,31),freq='1d')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [11]:
len(time)

365

# 保存数据

In [12]:
#new_NC.close()
new_NC = Dataset("D:/data/SST/SST_2021_day.nc", 'w', format='NETCDF4')

new_NC.createDimension('lat', len(lat))
new_NC.createDimension('lon', len(lon))
new_NC.createDimension('time', len(time))
#new_NC.createDimension('sst_mask', 1)

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'days since 2021-01-01'
time_var.long_name = 'days'
time_var.axis = 'T' 

new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))

new_NC.createVariable('sst', 'f',("time","lat","lon"))

#向变量中填充数据
new_NC.variables['lat'][:] = lat
new_NC.variables['lon'][:] = lon
new_NC.variables['time'][:] = np.array(time)

new_NC.variables['sst'][:]=np.array(sst)
 
#最后记得关闭文件
new_NC.close()

# 按月求平均值

In [13]:
sst.shape

(365, 3600, 7200)

In [14]:
import numpy as np
import calendar

# 假设data是您的数据，形状为 (4017, 720, 1440)
# 获取数据的形状
#1826       720        1440
num_days, num_rows, num_cols = sst.shape

# 创建一个数组来存储月平均值
monthly_averages_sst = []

start_day = 0 

for year in range(2021, 2022):  # 从2016年到2021年
    for month in range(1, 13):  # 1月到12月
        # 获取当前月份的天数，考虑年份和闰年
        days_in_month = calendar.monthrange(year, month)[1]
        # 在数据中选择当前月份的数据
        end_day = start_day + days_in_month
#         print(year,'年',month,'月')
#         print(start_day,end_day)
        monthly_data_sst = sst[start_day:end_day]
        # 计算每个月的平均值，并添加到结果数组中
        monthly_average_sst = np.nanmean(monthly_data_sst, axis=0)
    
        monthly_averages_sst.append(monthly_average_sst)
        
        start_day = end_day

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9028\2735911252.py:24: RuntimeWarning: Mean of empty slice
  monthly_average_sst = np.nanmean(monthly_data_sst, axis=0)


In [15]:
len(monthly_averages_sst)

12

## 获取时间

In [16]:
start_year = 2021
end_year =2021
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year,12,31),freq='1m')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [17]:
len(time)

12

# 保存月平均数据

In [18]:
np.array(monthly_averages_sst).shape

(12, 3600, 7200)

In [19]:
# new_NC.close()

In [19]:
#new_NC.close()
new_NC = Dataset("D:/data/SST/SST_2021_month.nc", 'w', format='NETCDF4')

new_NC.createDimension('lat', len(lat))
new_NC.createDimension('lon', len(lon))
new_NC.createDimension('time', len(time))


time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2021-01-15'
time_var.long_name = 'months'
time_var.axis = 'T' 

new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
new_NC.createVariable('sst', 'f',("time","lat","lon"))

#向变量中填充数据
new_NC.variables['lat'][:] = lat
new_NC.variables['lon'][:] = lon
new_NC.variables['time'][:] = np.array(time)
new_NC.variables['sst'][:]=np.array(monthly_averages_sst)
 
#最后记得关闭文件
new_NC.close()



In [34]:
adt_mask[0:1].shape

(1, 720, 1440)